In [1]:
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [3]:
model = load_model('Face_mask1.h5') 
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# model accept below hight and width of the image
img_width, img_hight = 224, 224 
 
cap = cv2.VideoCapture(0) 
img_count_full=0
font = cv2.FONT_HERSHEY_SIMPLEX 
org = (1,1)
class_lable=' '      
fontScale = 1 
color = (255, 0, 0) 
thickness = 2 
 
import tempfile
with tempfile.TemporaryDirectory() as directory:
    while True:
        img_count_full += 1

        responce, color_img = cap.read()
        if responce == False:
            break    

        gray_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray_img, 1.1, 6) # 1.1, 3) for 1.mp4

        img_count = 0
        for (x, y, w, h) in faces:
            org = (x-10,y-10)
            img_count +=1 
            color_face = color_img[y:y+h,x:x+w] # color face
            cv2.imwrite('/%s/%d%dface.jpg'%(directory,img_count_full,img_count),color_face)
            img = load_img('/%s/%d%dface.jpg'%(directory,img_count_full,img_count), target_size=(img_width,img_hight))

            img = img_to_array(img)/255
            img = np.expand_dims(img,axis=0)
            pred_prob = model.predict(img)
            pred=np.argmax(pred_prob)

            if pred==0:
                class_lable = "Mask "+str((pred_prob[0][0]*100).round(2))+"%"
                color = (0, 255, 0)
                cv2.rectangle(color_img, (x, y), (x+w, y+h), (255,0,0), 3)
                cv2.putText(color_img, class_lable, org, font,  
                                       fontScale, color, thickness, cv2.LINE_AA) 
            else:
                class_lable = "No Mask "+str((pred_prob[0][0]*100).round(2))+"%"
                color = (0,0,255)
                cv2.rectangle(color_img, (x, y), (x+w, y+h), (255,0,0), 3)
                cv2.putText(color_img, class_lable, org, font,  
                                       fontScale, color, thickness, cv2.LINE_AA) 

        cv2.imshow('LIVE face mask detection', color_img)

        if cv2.waitKey(1)==27:
            break
 
cap.release()
cv2.destroyAllWindows()